In [193]:
from IPython.display import display
from matplotlib import pyplot as plt
from sklearn import linear_model
import statsmodels.formula.api as smf

In [194]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math
import statistics
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from statsmodels.sandbox.regression.predstd import wls_prediction_std

df = pd.read_csv('Offenses_NYC_2014.csv')
df.isnull().values.any() 


True

In [195]:
features = ['Burglary','Murder and nonnegligent manslaughter', 'Robbery']
fx = df.loc[:, features].values

df = df.drop(df.index[227])
df = df.drop(df.index[363])


In [196]:
rob=df['Robbery']
pop=df['Population']
square=df['Population']**2
df['Population Squared']=square
murder=df['Murder and nonnegligent manslaughter']

In [197]:
count=0
rob=rob*1
for num in rob:
    if num > 0:
     rob[count]=1
    count +=1
df['rob']=rob

count=0
murder=murder*1
for num in murder:
    if num > 0:
     murder[count]=1
    count +=1
df['murder']=murder

burg=df['Burglary']
count=0
burg=burg*1
for num in burg:
    if num > 0:
     burg[count]=1
    count +=1
df['burglary']=burg

assault=df['Aggravated assault']
count=0
assault=assault*1
for num in assault:
    if num > 0:
     assault[count]=1
    count +=1
df['Agg. Assault']=assault

theft=df['Larceny-theft']
count=0
theft=theft*1
for num in theft:
    if num > 0:
     theft[count]=1
    count +=1
df['larceny']=theft

In [198]:
df.head()

City  Population  Violent crime  \
0             Adams Village      1851.0            0.0   
1  Addison Town and Village      2568.0            2.0   
2            Afton Village4       820.0            0.0   
3             Akron Village      2842.0            1.0   
4                   Albany4     98595.0          802.0   

   Murder and nonnegligent manslaughter  Rape (revised definition)  \
0                                   0.0                        NaN   
1                                   0.0                        NaN   
2                                   0.0                        0.0   
3                                   0.0                        NaN   
4                                   8.0                       54.0   

   Rape (legacy definition)2  Robbery  Aggravated assault  Property crime  \
0                        0.0      0.0                 0.0            11.0   
1                        0.0      1.0                 1.0            49.0   
2                        NaN      0.0                 0.0             1.0   
3                        0.0      0.0                 1.0            17.0   
4                        NaN    237.0               503.0          3888.0   

   Burglary  Larceny-theft  Motor vehicle theft  Arson3  Population Squared  \
0       1.0           10.0                  0.0     0.0        3.426201e+06   
1       1.0           47.0                  1.0     0.0        6.594624e+06   
2       0.0            1.0                  0.0     0.0        6.724000e+05   
3       0.0           17.0                  0.0     0.0        8.076964e+06   
4     683.0         3083.0                122.0    12.0        9.720974e+09   

   rob  murder  burglary  Agg. Assault  larceny  
0  0.0     0.0       1.0           0.0      1.0  
1  1.0     0.0       1.0           1.0      1.0  
2  0.0     0.0       0.0           0.0      1.0  
3  0.0     0.0       0.0           1.0      1.0  
4  1.0     1.0       1.0           1.0      1.0

In [199]:
regr = linear_model.LinearRegression()
Y = df['Property crime'].values.reshape(-1, 1)
X = df[['Population','Population Squared','rob','murder']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 1.56196460e-02  9.66047107e-08 -9.09739355e+00  8.25655952e+01]]

Intercept: 
 [7.29979152]

R-squared:
0.8887778996518514


In [200]:
#features = ['Burglary','Murder and nonnegligent manslaughter', 'Robbery']
#fx = df.loc[:, features].values
fx = StandardScaler().fit_transform(fx)


pca = PCA(n_components=2)
principalComponents = pca.fit_transform(fx)


In [201]:
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

df['principal_component_1']=principalDf['principal component 1']
df['principal_component_2']=principalDf['principal component 2']
#df.head(365)

In [202]:
regr = linear_model.LinearRegression()
Y = df['Property crime'].values.reshape(-1, 1)
X = df[['principal_component_1','principal_component_2']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 792.85976058 5627.23260999]]

Intercept: 
 [393.40878342]

R-squared:
0.9417844113557867


In [203]:
df.rename(index=str, columns={"Property crime": "PropertyCrime"},  inplace = True)
linear_formula = 'PropertyCrime ~ principal_component_1 + principal_component_2'

# Fit the model to our data using the formula.
lm = smf.ols(formula=linear_formula, data=df).fit()

In [204]:
lm.params

Intercept                 393.408783
principal_component_1     792.859761
principal_component_2    5627.232610
dtype: float64

In [205]:
lm.pvalues

Intercept                1.233791e-66
principal_component_1    2.343312e-07
principal_component_2    1.288094e-35
dtype: float64

In [206]:
lm.rsquared

0.9417844113557867